In [18]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [2]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.metrics import confusion_matrix

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

C:\Users\Gianni\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [9]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender"], axis=1) # "age"
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,1,1,211,1,50
1,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,149,1,40
2,27,1,0,0,0,1,1,1,1,1,...,1,1,1,1,0,1,1,168,1,28
3,29,1,1,0,1,1,1,1,1,0,...,1,1,1,0,0,1,0,230,1,37
4,19,1,0,0,0,1,1,0,1,1,...,0,0,1,0,0,0,1,361,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10413,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,243,1,24
10414,10,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,197,1,39
10415,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,447,2,33
10416,12,0,0,1,0,0,1,0,1,0,...,1,1,0,0,0,0,0,167,1,24


In [10]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.638 (+/- 0.014)


In [11]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

C:\Users\Gianni\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning:

The max_iter was reached which means the coef_ did not converge



,Actual,Predicted
5157,1,2
7580,1,2
4182,2,2
1207,2,2
5333,2,2
...,...,...
555,2,1
8497,2,2
8560,1,1
6252,2,2


,Questions,Male,Female,Other
0,Influence,0.067168,0.034728,-0.101895
1,Modesty,0.104793,-0.019485,-0.085308
2,Daring,-0.015241,0.035315,-0.020073
3,Confidence,0.061908,-0.123299,0.061391
4,Ruling,0.225517,-0.269618,0.044101
5,Responsibility,0.086836,-0.063202,-0.023634
6,IntrovExtro,-0.163535,0.168625,-0.005090
7,Success,0.022550,0.061288,-0.083838
8,Humble,0.093914,-0.014342,-0.079572
9,Leadership,0.009817,0.045568,-0.055385


In [23]:
# Using Statsmodels.api
# logit_model = sm.MNLogit(Y, X)
# result = logit_model.fit()
# print(result.summary2())

<IPython.core.display.Javascript object>

In [12]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[1307,  456,    0],
       [ 663,  676,    0],
       [  11,    2,    0]], dtype=int64)

### Multinomial Log Reg with Age Group

In [17]:
data = pd.read_csv("Data/data_AgeGroup.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "age", "Age_Group"], axis=1) #"gender"
Y = data["Age_Group"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age,Age_Group
0,18,0,0,0,1,0,0,0,0,1,...,0,0,0,0,1,1,211,1,50,5.0
1,6,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,149,1,40,4.0
2,27,1,0,0,0,1,1,1,1,1,...,1,1,1,0,1,1,168,1,28,3.0
3,29,1,1,0,1,1,1,1,1,0,...,1,1,0,0,1,0,230,1,37,4.0
4,19,1,0,0,0,1,1,0,1,1,...,0,1,0,0,0,1,361,1,27,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10413,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,243,1,24,2.0
10414,10,0,0,1,0,0,0,0,1,0,...,0,0,1,0,0,0,197,1,39,4.0
10415,6,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,447,2,33,3.0
10416,12,0,0,1,0,0,1,0,1,0,...,1,0,0,0,0,0,167,1,24,2.0


In [18]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

Mean Accuracy: 0.322 (+/- 0.017)


In [19]:
model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = [
    "Questions",
    "Under 18",
    "18-24",
    "25-34",
    "35-44",
    "45-54",
    "55-64",
    "65+",
]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
5157,6.0,2.0
7580,6.0,2.0
4182,4.0,5.0
1207,5.0,3.0
5333,5.0,4.0
...,...,...
555,3.0,5.0
8497,2.0,2.0
8560,3.0,5.0
6252,2.0,2.0


,Questions,Under 18,18-24,25-34,35-44,45-54,55-64,65+
0,Influence,-0.283191,-0.052847,0.037614,0.165057,0.125116,0.074611,-0.066360
1,Modesty,0.286681,0.054403,0.040833,0.057654,-0.014027,-0.009387,-0.416156
2,Daring,0.467429,0.301911,0.260189,0.061244,-0.068748,-0.195992,-0.826033
3,Confidence,0.208483,-0.138511,-0.290398,-0.166236,-0.225767,0.199576,0.412852
4,Ruling,-0.045565,0.088505,0.221479,-0.005046,-0.098697,-0.056124,-0.104553
5,Responsibility,0.351800,0.381575,0.227109,0.009586,-0.109369,-0.311522,-0.549179
6,IntrovExtro,0.047578,-0.161225,-0.064560,0.135053,-0.012481,0.044416,0.011219
7,Success,0.101466,0.475558,0.370876,0.076317,-0.102614,-0.170670,-0.750932
8,Humble,-0.139074,0.029404,0.253819,0.029901,-0.025563,-0.070091,-0.078395
9,Leadership,-0.309787,0.050430,0.146732,0.058781,0.152816,0.087318,-0.186290


In [20]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Under 18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+",],
    y=["Under 18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+",],
)
fig.update_xaxes(side="top")
fig.show()

array([[  0,  96,  44,  10,   2,   0,   0],
       [  4, 440, 288,  59,  33,   4,   2],
       [  1, 353, 361,  88,  50,  12,   1],
       [  3, 144, 203, 113,  77,   7,   1],
       [  0,  74, 132,  74, 104,   6,   2],
       [  0,  41,  77,  50,  76,   4,   2],
       [  0,   7,  22,  15,  30,   2,   1]], dtype=int64)

In [ ]:
data["Age_Group"].value_counts()